In [ ]:
import os
import sys
import glob
import numpy
import pandas
import tadpose

# mostly ploting
import ipywidgets
import seaborn as sns
from tqdm.auto import tqdm
from matplotlib import cm, colors
from matplotlib import pyplot as plt

# clustering and pca
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN, KMeans
from scipy.ndimage import gaussian_filter1d

### Basic definitions
* create a tadpole object
* configure alignment

In [ ]:
# main input required. SLEAP naysis file is expected to be in same folder with ending ".predictions.analysis.h5"
video_fn = "B:/fs3-bifexchgrp/BIF_StaffSci/Christoph/sweengrp/Juvenile_test/Wiping Recording bit/22-10-21_3-30pm_J11_wet2.mp4"

### Create tadpole and aligner

In [ ]:
# create Tadpole object
tadpole = tadpose.Tadpole.from_sleap(video_fn, bodyparts_cmap="tab20")

# create aligner by giving to part names and their correpsonding alignment location
aligner = tadpose.alignment.TadpoleAligner(
    {"Tail_Stem": numpy.array([0, 0.0]), "Heart_Center": numpy.array([0, 1.0])},
    scale=False,
)
tadpole.aligner = aligner

### Define skeleton of interest

In [ ]:
leg_parts = (
    "Right_toe",
    "Right_Foot",
    "Right_Ankle",
    "Right_Knee",
    "Right_Hip",
    "Tail_Stem",
    "Left_Hip",
    "Left_Knee",
    "Left_Ankle",
    "Left_Foot",
    "Left_toe",
)

In [ ]:
# get ego-centric locations
part_locs = tadpole.ego_locs(parts=leg_parts)

### Do PCA

In [ ]:
X = part_locs.reshape(part_locs.shape[0], -1)

# center and make unit variance
Xc = (X - X.mean(0)) / X.std(0)

# PCA with N components
N = 3

# Xp will contain the PCA components
pca = PCA(n_components=N)
pca.fit(Xc)
Xpca = pca.transform(Xc)

### Plot randomly selected skeltons with color-coded PC

In [ ]:
%matplotlib widget

f, ax = plt.subplots()
ax.set_aspect(1.0)

# select PC component to plot
pc = 0

norm = colors.Normalize(vmin=-5, vmax=5, clip=True)
for rand_ind in numpy.random.randint(X.shape[0], size=500):
    points = X[rand_ind, :]
    load = Xpca[rand_ind, pc]
    color = cm.seismic(norm(load))
    p = plt.plot(-points[::2].T, points[1::2].T, ".-", alpha=0.2, color=color)

plt.axis("off")
plt.title("Tracked leg points")

sm = plt.cm.ScalarMappable(cmap="seismic", norm=norm)
cbar = plt.colorbar(
    sm,
    fraction=0.033,
    pad=0.04,
)
cbar.ax.set_ylabel("PC 0")

### Smooth a little and smoothed compute gradient 

In [ ]:
# get first PCA component
pc0 = Xpca[:, 0:1]

# smooth
pc_s0 = tadpose.utils.smooth(pc0, win=5, poly=3, deriv=0)

# smooth diff
pc_s1 = tadpose.utils.smooth(pc0, win=5, poly=3, deriv=1)

### Interactive skelton viewer

In [ ]:
def show_skleton_viewer(tadpole, X, Xp, video_shape=(300, 400)):
    """
    Interactive viewer to visualize a skeleton from X together with scalar value e.g."Xp".
    Use left/right keys to go through time
    """

    plt.ioff()
    x_view = [-150, 150]
    y_view = [-150, 150]

    slider = ipywidgets.IntSlider(
        description="Time (frame)",
        value=0,
        min=0,
        max=X.shape[0] - 1,
        continuous_update=True,
        style={"min_width": 5000},
    )

    fig, axs = plt.subplots(1, 2, figsize=(8, 5))
    ax = axs[0]

    # which pc-component to vizualize
    pc = 0

    # normalize colors for that component
    norm = colors.Normalize(vmin=-5, vmax=5, clip=True)

    def update_lines(change):
        frame = change.new

        points = X[frame, :]
        pc_load = Xp[frame, pc]
        color = cm.jet(norm(pc_load))
        ax.clear()

        gray = tadpole.ego_image(
            frame, dest_height=video_shape[0], dest_width=video_shape[1], rgb=False
        )
        ax.imshow(
            gray,
            "gray",
            extent=(
                -gray.shape[1] // 2,
                gray.shape[1] // 2,
                -gray.shape[0] // 2,
                gray.shape[0] // 2,
            ),
        )

        ax.set_xticks([])
        ax.set_yticks([])

        ax.plot(points[::2].T, points[1::2].T, ".-", alpha=1, color=color)
        ax.set_xlim(x_view[0], x_view[1])
        ax.set_ylim(y_view[0], y_view[1])

        grad = Xp[:, 0]

        axs[1].clear()
        axs[1].plot(grad, "-", color="gray")
        axs[1].plot(frame, grad[frame], ".", color="red")
        axs[1].set_xlim(frame - 35, frame + 35)
        axs[1].set_ylabel("PC 0")
        axs[1].set_xlabel("Time (frames)")
        axs[1].set_xticks([frame - 30, frame, frame + 30])
        axs[1].set_xticklabels(list(map(str, [frame - 30, frame, frame + 30])))

        fig.canvas.draw()
        fig.canvas.flush_events()

    slider.observe(update_lines, names="value")
    slider.value = 1

    return ipywidgets.VBox([fig.canvas, slider])

In [ ]:
%matplotlib widget
show_skleton_viewer(tadpole, X, pc_s0)
# show_skleton_viewer(tadpole, X, pc_s1)